# Employee Salary Analytics - Feature Engineering

This notebook performs data cleaning and feature engineering on the employee salary dataset.

## Objectives:
1. Load raw dataset
2. Apply data cleaning functions
3. Create engineered features
4. Save cleaned and processed data


In [1]:
# Import necessary libraries
import sys
from pathlib import Path
import pandas as pd
import numpy as np

# Add src directory to path
project_root = Path().resolve().parent
sys.path.append(str(project_root / 'src'))

# Import custom modules
from load_data import load_raw_data, save_data
from clean_data import clean_dataset
from feature_engineering import (
    add_salary_per_hour, add_seniority_level, encode_categoricals
)

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)


## 1. Load Raw Data


In [2]:
# Load raw data
data_file_path = project_root / 'data' / 'raw' / 'employee_salaries.csv'
df_raw = load_raw_data(data_file_path)

print(f"\nInitial dataset shape: {df_raw.shape}")
print(f"Columns: {df_raw.columns.tolist()}")


✓ Loaded data from: /Users/aviyamegiddoshaked/employee-salary-analytics/data/raw/employee_salaries.csv
  Shape: 1200 rows × 17 columns
  Columns: Employee_ID, Name, Age, Gender, Country, City, Education, Job_Title, Department, Experience_Years, Salary_USD, Bonus_USD, Work_Hours_Per_Week, Remote_Work, Performance_Score, Joining_Year, Contract_Type
  Memory usage: 0.73 MB
  Data types:
Employee_ID             int64
Name                   object
Age                     int64
Gender                 object
Country                object
City                   object
Education              object
Job_Title              object
Department             object
Experience_Years        int64
Salary_USD              int64
Bonus_USD               int64
Work_Hours_Per_Week     int64
Remote_Work            object
Performance_Score       int64
Joining_Year            int64
Contract_Type          object
dtype: object

Initial dataset shape: (1200, 17)
Columns: ['Employee_ID', 'Name', 'Age', 'Gender', 'Cou

## 2. Data Cleaning

We apply comprehensive data cleaning to ensure data quality:
- Remove duplicates
- Handle missing values
- Filter invalid salary data
- Convert data types
- Standardize categorical values


In [3]:
# Apply cleaning functions
df_clean = clean_dataset(df_raw)

print(f"\nCleaned dataset shape: {df_clean.shape}")
print(f"Rows removed: {len(df_raw) - len(df_clean)}")


DATASET CLEANING SUMMARY

Initial dataset shape: 1200 rows × 17 columns
✓ No rows with missing salary found
✓ No rows with salary <= 0 found
✓ Converted experience_years to numeric
✓ Converted age to numeric
✓ Converted work_hours_per_week to numeric
✓ No duplicate rows found
✓ Converted name values to lowercase
✓ Converted gender values to lowercase
✓ Converted country values to lowercase
✓ Converted city values to lowercase
✓ Converted education values to lowercase
✓ Converted job_title values to lowercase
✓ Converted department values to lowercase
✓ Converted remote_work values to lowercase
✓ Converted contract_type values to lowercase

CLEANING COMPLETE
Final dataset shape: 1200 rows × 17 columns
Total rows removed: 0
Remaining missing values: 0


Cleaned dataset shape: (1200, 17)
Rows removed: 0


In [4]:
# Show example rows after cleaning
print("\nExample rows after cleaning:")
df_clean.head()



Example rows after cleaning:


,employee_id,name,age,gender,country,city,education,job_title,department,experience_years,salary_usd,bonus_usd,work_hours_per_week,remote_work,performance_score,joining_year,contract_type
0,1,employee_1,58,female,india,ny,bachelor,manager,it,0,130735,3650,41,yes,1,2015,part-time
1,2,employee_2,48,female,uk,ny,high school,designer,hr,4,54363,14600,52,no,2,2022,part-time
2,3,employee_3,34,male,usa,london,high school,manager,it,1,76694,15317,51,yes,8,2010,part-time
3,4,employee_4,27,male,uk,karachi,bachelor,developer,marketing,19,31582,19768,51,yes,5,2004,contract
4,5,employee_5,40,male,canada,london,master,manager,it,2,109910,18193,59,yes,7,2008,full-time


## 3. Feature Engineering

We create new features to improve model performance:
- **seniority_level**: Categorizes experience into junior/mid/senior
- **Encoded categoricals**: One-hot encoding for job_title, education, contract_type

**Note**: We do NOT include `salary_per_hour` in the modeling dataset because it would cause target leakage (salary_per_hour = salary / work_hours). This feature is only available for exploratory analysis purposes.


In [5]:
# Apply feature engineering
df_engineered = df_clean.copy()

# Note: We do NOT add salary_per_hour here to avoid target leakage
# salary_per_hour = salary / work_hours would leak target information

# Add seniority level
df_engineered = add_seniority_level(df_engineered)

# Encode categorical features
df_engineered = encode_categoricals(df_engineered)

print(f"\nEngineered dataset shape: {df_engineered.shape}")
print(f"New columns added: {df_engineered.shape[1] - df_clean.shape[1]}")


✓ Created seniority_level feature based on experience_years
✓ One-hot encoded 3 categorical columns: job_title, education, contract_type
  Added 5 new columns

Engineered dataset shape: (1200, 23)
New columns added: 6


In [6]:
# Show example rows after feature engineering
print("\nExample rows after feature engineering:")
df_engineered.head()



Example rows after feature engineering:


,employee_id,name,age,gender,country,city,department,experience_years,salary_usd,bonus_usd,work_hours_per_week,remote_work,performance_score,joining_year,seniority_level,job_title_designer,job_title_developer,job_title_manager,education_high school,education_master,education_phd,contract_type_full-time,contract_type_part-time
0,1,employee_1,58,female,india,ny,it,0,130735,3650,41,yes,1,2015,junior,0,0,1,0,0,0,0,1
1,2,employee_2,48,female,uk,ny,hr,4,54363,14600,52,no,2,2022,mid,1,0,0,1,0,0,0,1
2,3,employee_3,34,male,usa,london,it,1,76694,15317,51,yes,8,2010,junior,0,0,1,1,0,0,0,1
3,4,employee_4,27,male,uk,karachi,marketing,19,31582,19768,51,yes,5,2004,senior,0,1,0,0,0,0,0,0
4,5,employee_5,40,male,canada,london,it,2,109910,18193,59,yes,7,2008,junior,0,0,1,0,1,0,1,0


## 4. Save Processed Data

Save the cleaned and engineered dataset for use in modeling.


In [7]:
# Save to processed data directory
output_path = project_root / 'data' / 'processed' / 'salaries_clean.csv'
save_data(df_engineered, output_path)

print(f"\n✓ Data saved successfully!")
print(f"  Shape: {df_engineered.shape}")
print(f"  Location: {output_path}")

# Also save using pandas directly to ensure clean output
df_engineered.to_csv(output_path, index=False)
print(f"✓ Data overwritten to ensure clean output without salary_per_hour")


✓ Saved 1200 rows and 23 columns to /Users/aviyamegiddoshaked/employee-salary-analytics/data/processed/salaries_clean.csv

✓ Data saved successfully!
  Shape: (1200, 23)
  Location: /Users/aviyamegiddoshaked/employee-salary-analytics/data/processed/salaries_clean.csv
✓ Data overwritten to ensure clean output without salary_per_hour


## Summary

### Data Transformation Summary:
- Data has been cleaned and transformed through multiple stages
- Feature engineering has added new predictive features
- Processed data is saved and ready for modeling

### Key Features Created:
1. **seniority_level**: Experience-based categorization (junior/mid/senior)
2. **One-hot encoded features**: Binary features for categorical variables (job_title, education, contract_type)

### Important Note on Target Leakage:
- **salary_per_hour was intentionally excluded** from the modeling dataset
- This feature would cause target leakage since it's calculated as: salary_per_hour = salary / work_hours
- The `prepare_features()` function in `src/modeling.py` automatically excludes any salary-derived features
- Only legitimate predictors (age, experience, education, job_title, etc.) are used for modeling

The processed dataset is now ready for machine learning modeling without target leakage.
